<a href="https://colab.research.google.com/github/JohnMcKay/Road_Image_Classification/blob/master/McKay_Satellite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# System Imports
import os
import sys
from google.colab import drive  # to mount directory with .shp data

# 3rd Party Imports
import numpy as np
import tensorflow as tf
import scipy as sp
import h5py
import cv2
import matplotlib.pyplot as plt
import tqdm as tqdm
from sklearn.manifold import TSNE

In [0]:
drive.mount('/content/drive/')  # saved data location

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
# Globals
BATCH_SIZE = 64
EPOCHS = 15
DATA_PATH = '/content/drive/My Drive/sat_data/'  # has labels
IMAGE_PATH = os.path.join(DATA_PATH, 'image_tiles/')  # has images

In [0]:
# Definitions

# Keras Generators for Imagery
class Generator(tf.keras.utils.Sequence):
	def __init__(self, imNames, imLabels, batch_size, path_to_images, multi_task=False ):
		self.imNames = imNames
		self.path = path_to_images
		self.imLabels = imLabels
		self.batchSize = batch_size
		self.numIms = len(self.imNames)
		self.numSteps = self.numIms // batch_size
		if self.numIms % batch_size == 0:
			self.numSteps -= 1
		#
		self.n = 0
		self._multi_task = multi_task
	#
	def __len__(self):
		return self.numSteps
	#
	def steps_per_epoch(self):
		return self.numSteps
	#
	def __getitem__(self, index):
		batchSize = self.batchSize
		if index != self.numSteps:
			names = self.imNames[batchSize * index:batchSize * (index + 1)]
			labs = self.imLabels[batchSize * index:batchSize * (index + 1)]
		else:
			names = self.imNames[batchSize * index:]
			labs = self.imLabels[batchSize * index:]
		ims = []
		for i, n in enumerate(names):
			temp = cv2.imread(os.path.join(self.path, n))#
			temp = temp.astype('float32')
			temp -= temp.min()
			temp /= temp.max()
			ims.append(temp)
			# ims.append(np.flipud(temp))
		#
		ims = np.array(ims)
		if self._multi_task:
			temp = 1 - labs[:, 0] * 1.0
			return ims, {'a':labs, 'b':ims, 'c':temp}
		return ims, labs
	#
#

In [0]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Load Data
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Prepare data info
dataInfo = np.load(os.path.join(DATA_PATH, 'tileInfo.npy'))

# Get unique road labels
# https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
flatten = lambda l: [item for sublist in l for item in sublist]
undoLabels =  flatten([x[2].split('_') for x in dataInfo])
roadTypes = np.unique(undoLabels)
# Fix labels
roadTypes[0] = 'X'
# Note: there is only one case of 'T' so we throw it out
roadTypes = np.delete(roadTypes, -2)
numRoadTypes = len(roadTypes)

# Shuffle data entries
trainIndex = np.random.permutation(len(dataInfo))
dataInfo = dataInfo[trainIndex]

# Restate labels with categorical markers
strLabels = [x[2] if '' != x[2] else 'X' for x in dataInfo]  # handle empty
catLabels = np.array([[x.count(i) for i in roadTypes] for x in strLabels])  # categorical labels

# Set Training, Testing Parameters
trainPct = 0.75
testPct = 1 - trainPct
trainN = int(trainPct * len(dataInfo))
testN = int(testPct * len(dataInfo)) 

In [0]:
# Define Generators
allNames = [x[0] for x in dataInfo]  # isolates names
trainNames = [allNames[n] for n in range(trainN)]  # names for training
testNames = [allNames[n] for n in range(trainN, trainN + testN)]  # names for testing
trainGenerator = Generator(trainNames, catLabels[:trainN], BATCH_SIZE, IMAGE_PATH, multi_task=False)
testGenerator = Generator(testNames, catLabels[trainN:trainN + testN], BATCH_SIZE, IMAGE_PATH, multi_task=False)

In [0]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Model 1: Simple Network
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# mobilenet = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(256, 256, 3), 
#                                               include_top=True, 
#                                               weights=None, 
#                                               classes=numRoadTypes)
# inputLayer = tf.keras.layers.Input(shape=(256, 256, 3), dtype='uint8')
# lay = tf.keras.layers.Lambda(lambda x:tf.cast(x, tf.float32) / 255.0)(inputLayer)
# lay = mobilenet(lay)
# mnet = tf.keras.Model(inputLayer, lay)
# mnet.compile(loss='categorical_crossentropy',
#              optimizer=tf.keras.optimizers.Adam(lr=1e-6), 
#              metrics=['accuracy'])
inputLayer = tf.keras.Input(shape=(256, 256, 3))
mnet = tf.keras.layers.Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(inputLayer)
pix5 = tf.keras.layers.Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
mnet = tf.keras.layers.MaxPool2D()(mnet)
mnet = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
pix4 = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
mnet = tf.keras.layers.MaxPool2D()(mnet)
mnet = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
pix3 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
mnet = tf.keras.layers.MaxPool2D()(mnet)
mnet = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
pix2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
mnet = tf.keras.layers.MaxPool2D()(mnet)
mnet = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
pix1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
mnet = tf.keras.layers.Flatten()(pix1)
mnet = tf.keras.layers.Dense(64, activation='relu')(mnet)
pix0 = tf.keras.layers.Dense(64, activation='relu')(mnet)
mnet = tf.keras.layers.Dense(numRoadTypes, activation='sigmoid')(pix0)
mnet = tf.keras.models.Model(inputs=inputLayer, outputs=mnet)
mnet.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=1e-3, decay = 1/100), 
              metrics=['accuracy'])
mnet_history = mnet.fit_generator(generator=trainGenerator,
                                  validation_data=testGenerator,
                                  use_multiprocessing=True,
                                  workers=4,
                                  #class_weight={0:1/10, 1:1/5,2:1,3:1,4:1},
                                  epochs=EPOCHS)

Epoch 1/15
117/117 [==============================] - 85s 728ms/step - loss: 1.2896 - acc: 0.5888 - val_loss: 1.1254 - val_acc: 0.7284
Epoch 2/15
117/117 [==============================] - 79s 678ms/step - loss: 1.0559 - acc: 0.7377 - val_loss: 1.0232 - val_acc: 0.7396
Epoch 3/15
117/117 [==============================] - 85s 724ms/step - loss: 0.9817 - acc: 0.7551 - val_loss: 0.9960 - val_acc: 0.7572
Epoch 4/15
117/117 [==============================] - 79s 672ms/step - loss: 0.9542 - acc: 0.7595 - val_loss: 0.9633 - val_acc: 0.7608

Epoch 5/15
117/117 [==============================] - 79s 675ms/step - loss: 0.9137 - acc: 0.7686 - val_loss: 0.9501 - val_acc: 0.7628
Epoch 6/15
117/117 [==============================] - 79s 676ms/step - loss: 0.8760 - acc: 0.7798 - val_loss: 0.9575 - val_acc: 0.7552
Epoch 7/15
117/117 [==============================] - 79s 671ms/step - loss: 0.8570 - acc: 0.7804 - val_loss: 0.9330 - val_acc: 0.7680
Epoch 8/15
117/117 [==============================] - 

In [0]:
# serialize model to JSON
MNET_JSON = os.path.join(DATA_PATH, 'mnet_architecture.json')
model_json = mnet.to_json()
with open(MNET_JSON, "w") as json_file:
    json_file.write(model_json)
#
MNET_WEIGHTS = os.path.join(DATA_PATH, 'mnet_weights.h5')
mnet.save_weights(MNET_WEIGHTS)

AttributeError: ignored

In [0]:
pred= mnet.predict_generator(testGenerator, verbose=1)

39/39 [==============================] - 20s 508ms/step


In [0]:
pred

array([[2.7600825e-03, 7.3998052e-01, 7.4997544e-04, 2.5936484e-02,
        7.7234685e-02],
       [2.2026634e-01, 5.1943624e-01, 6.6832155e-02, 3.2718021e-01,
        2.9691809e-01],
       [2.0452738e-03, 9.2424130e-01, 4.0838420e-03, 2.8156519e-02,
        8.3711147e-03],
       ...,
       [1.5457192e-01, 2.4987388e-01, 7.9054028e-02, 2.3537338e-02,
        2.1994209e-01],
       [5.9605390e-02, 7.9750776e-01, 6.1488003e-02, 1.4671111e-01,
        1.1612466e-01],
       [9.5657176e-01, 1.7732382e-05, 2.1904707e-05, 8.9406967e-08,
        6.4054608e-02]], dtype=float32)

In [0]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Model 2: Multi Task Learning
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Base model
# root = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(256, 256, 3), 
#                                          include_top=True, 
#                                          weights=None, 
#                                          classes=numRoadTypes)
inputLayer = tf.keras.Input(shape=(256, 256, 3))
mnet = tf.keras.layers.Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(inputLayer)
pix5 = tf.keras.layers.Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
mnet = tf.keras.layers.MaxPool2D()(mnet)
mnet = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
pix4 = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
mnet = tf.keras.layers.MaxPool2D()(mnet)
mnet = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
pix3 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
mnet = tf.keras.layers.MaxPool2D()(mnet)
mnet = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
pix2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
mnet = tf.keras.layers.MaxPool2D()(mnet)
mnet = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
pix1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.he_normal())(mnet)
mnet = tf.keras.layers.Flatten()(pix1)
mnet = tf.keras.layers.Dense(64, activation='relu')(mnet)
pix0 = tf.keras.layers.Dense(64, activation='relu')(mnet)
mnet = tf.keras.layers.Dense(numRoadTypes, activation='sigmoid', name='a')(pix0)

# Autoencoder network
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(pix1)
#
x = tf.keras.layers.UpSampling2D((2, 2))(x)  # 32, 32
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.concatenate([x, pix2], axis=3)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
#
x = tf.keras.layers.UpSampling2D((2, 2))(x)  # 64, 64
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.concatenate([x, pix3], axis=3)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
#
x = tf.keras.layers.UpSampling2D((2, 2))(x)  # 128, 128
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.concatenate([x, pix4], axis=3)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
#
x = tf.keras.layers.UpSampling2D((2, 2))(x)  # 256, 256
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.concatenate([x, pix5], axis=3)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
#
x = tf.keras.layers.Conv2D(3, (3, 3), activation='relu', padding='same', name='b')(x)

# Road-or-not network
y = tf.keras.layers.Dense(1, activation='softmax', name='c')(pix0)

ins = tf.keras.Input(shape=(256, 256, 3))
outManyClass = mnet
outReconstruct = x
outOneClass = y 

# First train up model for an epoch
net0 = tf.keras.Model(inputs=inputLayer, outputs=outManyClass)
net0.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=tf.keras.optimizers.Adam(lr=1e-3, decay=1/6000))
net0.fit_generator(generator=trainGenerator, validation_data=testGenerator, use_multiprocessing=True, workers=4, epochs=1)

# Aggregate Model
net = tf.keras.Model(inputs=inputLayer, outputs=[outManyClass, outReconstruct, outOneClass])
net.compile(loss={'a':'categorical_crossentropy', 'b':'mse', 'c':'binary_crossentropy'},
            optimizer=tf.keras.optimizers.Adam(lr=1e-3, decay=1/400), 
            metrics = ['accuracy'], 
            loss_weights = [1, 1e-1, 1e-2])  # play with weights
trainGenerator = Generator(trainNames, catLabels[:trainN], BATCH_SIZE // 4, IMAGE_PATH, True)
testGenerator = Generator(testNames, catLabels[trainN:trainN + testN], BATCH_SIZE // 4, IMAGE_PATH, True)
net_history = net.fit_generator(generator=trainGenerator,
                                validation_data=testGenerator,
                                steps_per_epoch=len(trainGenerator),
                                use_multiprocessing=True,
                                workers=6,
                                epochs=EPOCHS)

468/468 [==============================] - 91s 195ms/step - loss: 1.0918 - acc: 0.7141 - val_loss: 0.9625 - val_acc: 0.7576
Epoch 1/15
468/468 [==============================] - 124s 264ms/step - loss: 1.0330 - a_loss: 0.9872 - b_loss: 0.0133 - c_loss: 4.4497 - a_acc: 0.7532 - b_acc: 0.5739 - c_acc: 0.7098 - val_loss: 1.0569 - val_a_loss: 1.0107 - val_b_loss: 0.0033 - val_c_loss: 4.5821 - val_a_acc: 0.7364 - val_b_acc: 0.6606 - val_c_acc: 0.6995
Epoch 2/15
468/468 [==============================] - 116s 248ms/step - loss: 0.9619 - a_loss: 0.9172 - b_loss: 0.0025 - c_loss: 4.4497 - a_acc: 0.7724 - b_acc: 0.6848 - c_acc: 0.7098 - val_loss: 0.9726 - val_a_loss: 0.9266 - val_b_loss: 0.0020 - val_c_loss: 4.5821 - val_a_acc: 0.7692 - val_b_acc: 0.7206 - val_c_acc: 0.6995
Epoch 3/15
468/468 [==============================] - 117s 251ms/step - loss: 0.9093 - a_loss: 0.8647 - b_loss: 0.0017 - c_loss: 4.4497 - a_acc: 0.7869 - b_acc: 0.7203 - c_acc: 0.7098 - val_loss: 0.9654 - val_a_loss: 0.9195 

In [0]:
NET_JSON = os.path.join(DATA_PATH, 'net_architecture.json')
model_json = net.to_json()y
with open(NET_JSON, "w") as json_file:
    json_file.write(model_json)
#
NET_WEIGHTS = os.path.join(DATA_PATH, 'net_weights.h5')
net.save_weights(NET_WEIGHTS)

In [0]:
pred = net.predict_generator(testGenerator, steps=len(testGenerator), verbose=1)

NameError: ignored